## UN Comtrade (Comtrade+) -> features mensais para Ureia (HS 310210)

Features típicas:
- China exportações (oferta)
- Índia importações (demanda / tenders)
- (opcional) Brasil importações (proxy de demanda local)

Fonte/API:
- Data API endpoint: https://comtradeapi.un.org/data/v1/get/C/{freq}/HS  (freq: A ou M)
- Anal usa period=`YYYY` e pode filtrar flowCode (`X=Export`, `M=Import`)

## Importando principais bibliotecas
---

In [ ]:
import os
import time
from dotenv import load_dotenv
from dataclasses import dataclass
from pathlib import Path
from typing import Iterable, Optional, List

import comtradeapicall
import pandas as pd
import requests

load_dotenv() # Carrega variáveis de ambiente do arquivo .env

### Configurações
---

In [ ]:
def _safe_mkdir(path: str) -> None:
    os.makedirs(path, exist_ok=True)

CHN = 156
IND = 356
BRA = 76
WORLD = 0

HS_UREA = "310210"
DATA_DIR = os.path.abspath("data")
COMTRADE_DIR = os.path.join(DATA_DIR, "comtrade")

_safe_mkdir(DATA_DIR)
_safe_mkdir(COMTRADE_DIR)

@dataclass(frozen=True)
class ComtradeQuery:
    reporter_code: int
    partner_code: int = WORLD
    cmd_code: str = HS_UREA
    freq: str = "M"  # "A" ou "M"
    flow_code: Optional[str] = None  # "X" (export) ou "M" (import) ou None (ambos)
    customs_code: str = "C00"
    mot_code: int = 0
    partner2_code: int = 0
    include_desc: bool = True
    max_records: int = 5000  # suficiente p/ 1 cmd + world + 1 reporter

In [ ]:
def fetch_wits_data(reporter: str, product: str, start_year: int, end_year: int, flow: str = 'M') -> pd.DataFrame:
    """
    Baixa dados de comércio da WITS API para um produto específico.
    
    Parâmetros:
    - reporter: Código ISO3 do país (e.g., 'IND' para Índia)
    - product: Código HS do produto (e.g., '310210' para Ureia)
    - start_year: Ano inicial
    - end_year: Ano final
    - flow: 'M' para import, 'X' para export
    
    Retorna DataFrame com os dados.
    """
    base_url = "https://wits.worldbank.org/API/V1/wits/WITSApiService.svc/data"
    dfs = []
    
    trade_flow = 'Import' if flow == 'M' else 'Export'
    
    for year in range(start_year, end_year + 1):
        params = {
            'Reporter': reporter,
            'Partner': 'WLD',  # World
            'Product': product,
            'Year': str(year),
            'TradeFlow': trade_flow,
            'Format': 'JSON'
        }
        
        try:
            response = requests.post(base_url, params=params, timeout=30)
            response.raise_for_status()
            data = response.json()
            
            if data and isinstance(data, list) and len(data) > 0:
                df = pd.DataFrame(data)
                dfs.append(df)
            else:
                print(f"Nenhum dado para {year}")
                
        except Exception as e:
            print(f"Erro ao buscar dados para {year}: {e}")
    
    if dfs:
        return pd.concat(dfs, ignore_index=True)
    else:
        return pd.DataFrame()